<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/setfit_reference_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
dataset = load_dataset("ethos", "multilabel")

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'violence', 'directed_vs_generalized', 'gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation'],
        num_rows: 433
    })
})

In [14]:
dataset['train'][1:10]

{'text': ['You look like Sloth with deeper Down’s syndrome',
  'You look like Russian and speak like Indian. Both are disgusting go kill yourself',
  'Women deserve to be abused, I guess.',
  'Women are made for making babies and cooking dinner and nothing else!!!',
  "why haven't you faggots taken the bi-pill yet",
  'Why don’t you go beck your god to murder you? Christianity is a joke',
  'why does that african american monkey hold gun like that?',
  "Why does Bulgaria exist as a country? It's one of those homosexual ones.",
  'Why do negroes tear down their own neighborhoods when a black thug gets killed by law enforcement? Must be because of low IQ.'],
 'violence': [0, 1, 1, 0, 1, 1, 0, 0, 0],
 'directed_vs_generalized': [1, 1, 0, 0, 1, 1, 1, 1, 0],
 'gender': [0, 0, 1, 1, 0, 0, 0, 0, 0],
 'race': [0, 0, 0, 0, 0, 0, 1, 0, 1],
 'national_origin': [0, 1, 0, 0, 0, 0, 0, 0, 0],
 'disability': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'religion': [0, 0, 0, 0, 0, 1, 0, 0, 0],
 'sexual_orientation': 

# get list of str features

In [4]:
import numpy as np

features = dataset["train"].column_names
features.remove("text")
features

['violence',
 'directed_vs_generalized',
 'gender',
 'race',
 'national_origin',
 'disability',
 'religion',
 'sexual_orientation']

In [5]:
type(features)

list

In [6]:
num_samples = 8
samples = np.concatenate(
    [np.random.choice(np.where(dataset["train"][f])[0], num_samples) for f in features]
)

Output array with 64 elements 

In [17]:
#len(samples)
samples

array([ 73, 252,  63, 230,  57, 193, 298, 212,  57, 340, 318, 298, 155,
       270, 155, 172, 377, 285, 366, 188, 190, 418, 375, 423, 210, 302,
       199,  74,  48, 302,   7,  24, 304, 252, 238, 114,  32, 412, 357,
       235, 292, 208, 203,  39, 288, 280, 175, 180, 158,  99, 317, 129,
        37, 211,  37, 216, 176, 148, 153, 147,  33,  60, 337, 427])

In [16]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}
dataset = dataset.map(encode_labels)

  0%|          | 0/433 [00:00<?, ?ex/s]

In [25]:
dataset['train']['labels'][1:10] # as integer all scores single labels column 

[[0, 1, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0]]

In [26]:
train_dataset = dataset["train"].select(samples)
eval_dataset = dataset["train"].select(
    np.setdiff1d(np.arange(len(dataset["train"])), samples)
)

In [ ]:
!pip install setfit

In [ ]:
from setfit import SetFitModel
model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

In [31]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=2,
    column_mapping={"text": "text", "labels": "label"},
)

In [32]:
trainer.train()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 364
  Num epochs = 1
  Total optimization steps = 23
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

KeyboardInterrupt: ignored